In [17]:
import pandas as pd
import numpy as np
import json
import matplotlib.pyplot as plt
import seaborn as sns
import ast


In [18]:
# Ustawienie, aby widzieć więcej kolumn
pd.set_option('display.max_columns', 50)

# Wczytaj surowe dane
df_raw = pd.read_csv('../data/raw/all_matches.csv')

df_raw.head()

,date,event,eventobj,game,id,offline,pla,plb,rca,rcb,resource_uri,rta,rtb,sca,scb,treated
0,2023-12-17,NaN,{'fullname': 'ESL Pro Tour 2023/24 SC2 Masters...,LotV,452881,True,"{'country': 'FR', 'id': 5878, 'race': 'T', 're...","{'country': 'FI', 'id': 485, 'race': 'Z', 'res...",T,Z,/api/v1/match/452881/,"{'decay': 0, 'dev': 0.0702217126668418, 'dev_v...","{'decay': 0, 'dev': 0.0819425378669093, 'dev_v...",3,1,True
1,2025-01-25,NaN,{'fullname': 'LiuLi Cup 2024 Grand Finals Main...,LotV,473920,False,"{'country': 'FI', 'id': 485, 'race': 'Z', 'res...","{'country': 'FR', 'id': 5878, 'race': 'T', 're...",Z,T,/api/v1/match/473920/,"{'decay': 0, 'dev': 0.0936552902980403, 'dev_v...","{'decay': 0, 'dev': 0.0696930565755874, 'dev_v...",2,3,True
2,2021-08-15,NaN,{'fullname': 'ESL Pro Tour 2021/22 DH SC2 Mast...,LotV,392760,False,"{'country': 'FI', 'id': 485, 'race': 'Z', 'res...","{'country': 'FR', 'id': 5878, 'race': 'T', 're...",Z,T,/api/v1/match/392760/,"{'decay': 0, 'dev': 0.11268374277312, 'dev_vp'...","{'decay': 0, 'dev': 0.0786558146335471, 'dev_v...",1,4,True
3,2019-05-17,NaN,{'fullname': 'WCS 2019 Spring Main Tournament ...,LotV,310756,True,"{'country': 'FI', 'id': 485, 'race': 'Z', 'res...","{'country': 'FR', 'id': 5878, 'race': 'T', 're...",Z,T,/api/v1/match/310756/,"{'decay': 0, 'dev': 0.078536597469805, 'dev_vp...","{'decay': 0, 'dev': 0.0712631294222908, 'dev_v...",2,1,True
4,2022-10-11,NaN,{'fullname': 'ESL Pro Tour 2022/23 DH SC2 Mast...,LotV,423599,False,"{'country': 'FI', 'id': 485, 'race': 'Z', 'res...","{'country': 'FR', 'id': 5878, 'race': 'T', 're...",Z,T,/api/v1/match/423599/,"{'decay': 0, 'dev': 0.0924126594744766, 'dev_v...","{'decay': 0, 'dev': 0.0728103220395189, 'dev_v...",3,1,True


In [19]:
# 1. Zostaw tylko mecze, które wpłynęły na ranking
df_clean = df_raw[df_raw['treated'] == True].copy()

# 2. Usuń kolumny bezużyteczne dla modelu
kolumny_do_usuniecia = ['event', 'game', 'id', 'resource_uri', 'treated']
df_clean = df_clean.drop(columns=kolumny_do_usuniecia)

In [20]:
df_clean['rta'] = df_clean['rta'].apply(ast.literal_eval)
df_clean['rtb'] = df_clean['rtb'].apply(ast.literal_eval)
df_clean['pla'] = df_clean['pla'].apply(ast.literal_eval)
df_clean['plb'] = df_clean['plb'].apply(ast.literal_eval)

# Teraz, gdy to są słowniki, możesz wyciągnąć z nich dane
# pandas.json_normalize to potężne narzędzie, ale apply(pd.Series) jest czasem prostsze

# Przykład wyciągnięcia ratingów:
df_clean['pla_id'] = df_clean['pla'].apply(lambda x: x['id'])
df_clean['plb_id'] = df_clean['plb'].apply(lambda x: x['id'])

df_clean['rta_rating'] = df_clean['rta'].apply(lambda x: x['rating'])
df_clean['rtb_rating'] = df_clean['rtb'].apply(lambda x: x['rating'])

# Wyciągnij ratingi matchupowe (najważniejsze!)
df_clean['rta_vP'] = df_clean['rta'].apply(lambda x: x['tot_vp'])
df_clean['rta_vT'] = df_clean['rta'].apply(lambda x: x['tot_vt'])
df_clean['rta_vZ'] = df_clean['rta'].apply(lambda x: x['tot_vz'])

df_clean['rtb_vP'] = df_clean['rtb'].apply(lambda x: x['tot_vp'])
df_clean['rtb_vT'] = df_clean['rtb'].apply(lambda x: x['tot_vt'])
df_clean['rtb_vZ'] = df_clean['rtb'].apply(lambda x: x['tot_vz'])

# Po wyciągnięciu danych, możesz usunąć oryginalne, skomplikowane kolumny
df_clean = df_clean.drop(columns=['rta', 'rtb', 'pla', 'plb', 'eventobj'])

ValueError: malformed node or string: nan